In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate


llm = ChatOpenAI(
    temperature=0.1,
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=500,
    return_messages=True,
)

examples = [
    {
        "movie": "Top Gun",
        "emoji": "🛩️👨‍✈️🔥"
    },
    {
        "movie": "The Godfather",
        "emoji": "👨‍👨‍👦🔫🍝"
    },
    {
        "movie": "La La Land",
        "emoji": "💃🎹🌇"
    },
    {
        "movie": "Jurassic Park",
        "emoji": "🦖🌴⚡"
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "이 {movie}를 이모티콘 3개로 표현하세요."),
        ("ai", "{emoji}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 영화 평론가이자 대화 파트너입니다. "
            "사용자가 영화 제목을 말하면 이모티콘 3개로 표현하세요."
            "그 외의 질문에는 대화 맥락에 따라 자연스럽게 응답하세요. 이모티콘을 사용하지 않아도 됩니다."
            "상황에 맞는 방식으로 지혜롭게 답하세요.",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{movie}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(movie):
    result = chain.invoke({"movie": movie})
    memory.save_context(
        {"input": movie},
        {"output": result.content},
    )
    print(result)

In [2]:
invoke_chain("Avatar")

content='🎬🎥🌿'


In [3]:
invoke_chain("인사이드아웃")

content='😊💔🧠'


In [4]:
memory.load_memory_variables({}) 

{'history': [HumanMessage(content='Avatar'),
  AIMessage(content='🎬🎥🌿'),
  HumanMessage(content='인사이드아웃'),
  AIMessage(content='😊💔🧠')]}

In [6]:
invoke_chain("내가 첫번쨰로 물어본 영화는?")

content='첫 번째로 물어보신 영화는 "Avatar"였어요. 그 영화는 시각적으로 놀라운 작품이죠!'


In [7]:
invoke_chain("터미네이터")


content='🤖🔫🔥'


In [8]:
invoke_chain("내가 지금까지 물어봤던 모든 영화는?")

content='지금까지 물어보신 영화는 "Avatar", "인사이드아웃", "터미네이터"였어요. 모두 다양한 장르의 인기 있는 영화들이에요!'
